# Unit 5 - Financial Planning

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import time 
import seaborn as sns

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
# YOUR CODE HERE!
crypto_holdings = {"btc" : 1.2, "eth" : 5.3}
crypto_tickers = ["BTC, ETH"]

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

json_btc = requests.get(btc_url).json()
json_eth = requests.get(eth_url).json()

In [ ]:
# Fetch current BTC price
# YOUR CODE HERE!
btc_price = json_btc["data"]["1"]["quotes"]["CAD"]["price"]

# Fetch current ETH price
# YOUR CODE HERE!
eth_price = json_eth["data"]["1027"]["quotes"]["CAD"]["price"]

# Compute current value of my crpto
# YOUR CODE HERE!
my_btc = btc_price * crypto_holdings["btc"]
my_eth = eth_price * crypto_holdings["eth"]

# Compute current crypto wallet balance
crypto_wallet = my_btc + my_eth


# print(f"The current value of your {btc_amount} BTC is ${btc_value:0.2f}")

print(f"The current value of your {crypto_holdings['btc']} BTC is ${my_btc:0.2f}")
print(f"The current value of your {crypto_holdings['eth']} ETH is ${my_eth:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
# YOUR CODE HERE!
alpaca_key_id = os.getenv("ALPACA_KEY_ID")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
# YOUR CODE HERE!
alpaca = tradeapi.REST(
    alpaca_key_id,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Format current date as ISO format
# YOUR CODE HERE!
today = pd.Timestamp("2000-06-11", tz="Canada/Central").isoformat()
end = pd.Timestamp("2020-06-11", tz="Canada/Central").isoformat()
# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
# YOUR CODE HERE!
stock_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    limit = 1000
).df


# Preview DataFrame
# YOUR CODE HERE!
# stock_portfolio.sort_index(ascending=False, inplace=True)
stock_portfolio

In [ ]:
# Pick AGG and SPY close prices
# YOUR CODE HERE!

# Get closing prices
agg_close_price = stock_portfolio["AGG"]["close"][1000]
spy_close_price = stock_portfolio["SPY"]["close"][1000]


# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
# YOUR CODE HERE!
my_spy_value = spy_close_price * my_spy
my_agg_value = agg_close_price * my_agg

shares_wallet = my_spy + my_agg

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")
shares_wallet = my_spy_value + my_agg_value

### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

monthly_income = 12000 

# Consolidate financial assets data
# YOUR CODE HERE!
savings_dict = {"crypto" : [crypto_wallet],  
           "shares" : [shares_wallet]
                                 }

# Create an assests DataFrame
# YOUR CODE HERE!

df_savings = pd.DataFrame (savings_dict, columns = ['crypto', 'shares']).round(2)

# Display savings DataFrame
print('Savings')
df_savings

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!
####################################


In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!
savings = crypto_wallet + shares_wallet

# Validate saving health
# YOUR CODE HERE!
if emergency_fund < savings:
    print("Congratulations! You have enough money in your emergency fund.")
else: 
    print("abs(savings-emergency_fund)")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
# YOUR CODE HERE!
stock_portfolio_5y = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    limit = 1000
).df


# Display sample data
stock_portfolio_5y.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

# Printing the simulation input data=
num_sims = 500 

MC_STOCKS = MCSimulation(
    portfolio_data = stock_portfolio_5y,
    num_simulation = num_sims,
    num_trading_days = 252 * 30,
)

# Set a limit for the simulation
limit= 1000

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!
MC_STOCKS.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!
line_plot = MC_STOCKS.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!
distribution_plot = MC_STOCKS.lineplot()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!
tbl = MC_STOCKS.summarize_cumulative_return()

# Print summary statistics
print(tbl)


### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!
ci_lower = round(tbl[8]*initial_investment,2)
ci_upper = round(tbl[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
increase_initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!
ci_lower = round(tbl[8]*increase_initial_investment,2)
ci_upper = round(tbl[9]*increase_initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${increase_initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!
MC_STOCKS_2 = MCSimulation(
    portfolio_data = stock_portfolio_5y,
    num_simulation = num_sims,
    num_trading_days = 252 * 5,
)

#  Set limit
limit = 1000

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!
MC_STOCKS_2.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!
line_plot = MC_STOCKS_2.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!
tbl = MC_STOCKS_2.summarize_cumulative_return()

# Print summary statistics
print(tbl)


In [ ]:
# Set initial investment
# YOUR CODE HERE!
retirement_investment = 50000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!
ci_lower_five = round(tbl[8]*retirement_investment,2)
ci_upper_five = round(tbl[9]*retirement_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${retirement_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!
MC_STOCKS_3 = MCSimulation(
    portfolio_data = stock_portfolio_5y,
    num_simulation = num_sims,
    num_trading_days = 252 * 10,
)

#  Set limit
limit = 1000

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!
MC_STOCKS_3.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!
line_plot = MC_STOCKS_3.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!
tbl = MC_STOCKS_3.summarize_cumulative_return()

# Print summary statistics
# YOUR CODE HERE!
print(tbl)


In [ ]:
# Set initial investment
# YOUR CODE HERE!
initial_10yr_investment = 100000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!
ci_lower_10 = round(tbl[8]*initial_10yr_investment,2)
ci_upper_10 = round(tbl[9]*initial_10yr_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_10yr_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_10} and ${ci_upper_10}")